<a href="https://colab.research.google.com/github/pleexik/AI_labs/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 1. Базовое использование LLM

### Инициализируем модель

Qwen2 — новая серия больших языковых моделей Qwen. Для Qwen2 существует ряд базовых языковых моделей и языковых моделей с настройкой инструкций с диапазоном от 0,5 до 72 миллиардов параметров, включая модель «Смесь экспертов». Мы будем использовать настроенную по инструкциям модель 1.5B Qwen2.

По сравнению с современными языковыми моделями с открытым исходным кодом, включая ранее выпущенный Qwen1.5, Qwen2 в целом превзошел большинство моделей с открытым исходным кодом и продемонстрировал конкурентоспособность по сравнению с проприетарными моделями в ряде тестов, нацеленных на понимание языка, создание языков, многоязычные возможности. , кодирование, математика, рассуждения и т. д.

Подробнее с моделью Qwen можно ознакомиться по ссылке https://huggingface.co/Qwen/Qwen2-1.5B-Instruct

In [ ]:
%pip install torch>=2.2.2

In [ ]:
%pip install transformers

In [ ]:
%pip install accelerate

In [ ]:
%pip install nltk

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

CUDA available: True
CUDA version: 12.6


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    device_map=device,
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Проверяем работу модели

In [ ]:
prompt = "Коротко расскажи мне о Больших языковых моделях (LLM)."
messages = [
    {"role": "system", "content": "Ты AI помощник Qwen."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Большие Языковые Модели (LLMs) - это искусственные интеллектуальные системы, которые способны понимать и интерпретировать человеческое язычество на более высоком уровне, чем это возможно с обычными машинами. Они используются для решения сложных задач, таких как анализ текста, обнаружение структур в данных, распознавание лиц, перевод языка и многое другое.

Основная цель LLM-систем - обучить себя на основе большого объема данных, чтобы они могли "читать" и "понимать" человеческий язык, а не просто выдавать предсказания или ответы. Это позволяет им работать над сложными задачами, требующими глубокого анализа и понимания контекста. 

Примеры известных больших языковых моделей включают Google's Transformer модели, BERT, GPT, T5 и другие.


### 1. Базовый инференс

Напишите функцию, которая будет принимать модель, токенизатор, а также список сообщений пользователя и модели, и возвращать ответ модели в виде текста.

Сообщения должны подаваться в следующем виде
```
chat = [
   {"role": "user", "content": ...},
   {"role": "assistant", "content": ...},
   {"role": "user", "content": ...},
]
```

Полезные функции
https://huggingface.co/docs/transformers/main/en/chat_templating
https://huggingface.co/docs/transformers/en/main_classes/tokenizer

Добавьте `max_new_tokens`, `temperature`, `repetition_penalty`, `no_repeat_ngram_size` в качестве параметров этой функции. Протестируйте работу модели на различных значениях этих параметров.

In [ ]:
def generate_response(model, tokenizer, messages, max_new_tokens=512, temperature=0.7,
                     repetition_penalty=1.1, no_repeat_ngram_size=3):
    """
    Генерирует ответ модели на основе истории диалога
    """

    # Применяем чат-шаблон для форматирования сообщений
    formatted_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Токенизируем входные данные и перемещаем на устройство модели
    inputs = tokenizer.encode(formatted_input, return_tensors="pt")

    # Получаем устройство модели и перемещаем входные данные
    device = next(model.parameters()).device
    inputs = inputs.to(device)

    # Генерируем ответ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Декодируем только сгенерированную часть (исключая промпт)
    generated_tokens = outputs[0][inputs.shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response
    # Тестовый диалог
test_chat = [
    {"role": "user", "content": "Привет! Объясни мне почему 2+2=4, а не 5 или 10"},
]

print("=== Тестирование различных параметров ===\n")

# 1. Стандартные параметры
print("1. Стандартные параметры:")
response1 = generate_response(
    model, tokenizer, test_chat,
    max_new_tokens=250,
    temperature=0.7,
    repetition_penalty=1.1
)
print(f"Ответ: {response1}\n")

# 2. Низкая температура (более детерминированные ответы)
print("2. Низкая температура (temperature=0.3):")
response2 = generate_response(
    model, tokenizer, test_chat,
    max_new_tokens=200,
    temperature=0.3,
    repetition_penalty=1.1
)
print(f"Ответ: {response2}\n")

# 3. Высокая температура (более креативные ответы)
print("3. Высокая температура (temperature=1.2):")
response3 = generate_response(
    model, tokenizer, test_chat,
    max_new_tokens=200,
    temperature=1.2,
    repetition_penalty=1.1
)
print(f"Ответ: {response3}\n")

# 4. Сильный штраф за повторения
print("4. Сильный штраф за повторения (repetition_penalty=1.5):")
response4 = generate_response(
    model, tokenizer, test_chat,
    max_new_tokens=200,
    temperature=0.7,
    repetition_penalty=1.5
)
print(f"Ответ: {response4}\n")

=== Тестирование различных параметров ===

1. Стандартные параметры:
Ответ: Это математическая операция и ее значение определяется формулой. 

"2 + 2 = 4" - это упрощенная формула для сложения двух чисел равных двум (2). Это означает, что когда мы сложаем два числа, равные дву-м, результат будет равно четырем.

Однако, если вы говорите "2 + x = 5", где "x" - может быть любое число, то ответ будет 3 (когда вы добавляете 3 к 2, получается 5), так как это тоже является упростленной формулами для слагания чисел.

Если же говорите о "2 * 2 + y = 6", где y - может принимать любое значение, то здесь результат будет 4, потому что при сложении чисел 2 и 2 мы получаем 4 и затем добавляем в это еще одно число, которое и есть "y". Так что 2*2 +y = 2(2) + y= 4 + y.

2. Низкая температура (temperature=0.3):
Ответ: Это математическая константа, известная как "один". Это означает, что в математуре, когда мы говорим об операциях (+, -, *, /), результат всегда будет равен единице, если мы делаем это пра

### 2. Суммаризация небольших текстов

До появления LLM задача суммаризации текстов, то есть их краткого пересказа с сохранением смысла считалась сложной задачей. Вот пример из одного курса 2020 года
```
... модели Transformer могут быть использованы для сжатия длинных документов в краткое изложение - задача, известная как суммаризация текста (text summarization). Это одна из самых сложных задач NLP, поскольку она требует целого ряда способностей, таких как понимание длинных отрывков и генерация связного текста, отражающего основные темы документа...
```

Теперь же, с появлением больших языковых моделей суммаризация текста является одной из естественных задач, для которой их применяют. Давайте попробуем суммаризовать отрывок из книги с помощью Qwen2.

In [ ]:
with open('Толстый и тонкий.txt') as f:
    novel = f.read()

print(novel)

Антон Чехов
ТОЛСТЫЙ И ТОНКИЙ
На вокзале Николаевской железной дороги встретились два приятеля: один толстый, другой тонкий. Толстый только что пообедал на вокзале, и губы его, подернутые маслом, лоснились, как спелые вишни. Пахло от него хересом и флер-д'оранжем. Тонкий же только что вышел из вагона и был навьючен чемоданами, узлами и картонками. Пахло от него ветчиной и кофейной гущей. Из-за его спины выглядывала худенькая женщина с длинным подбородком — его жена, и высокий гимназист с прищуренным глазом — его сын.
— Порфирий! — воскликнул толстый, увидев тонкого.— Ты ли это? Голубчик мой! Сколько зим, сколько лет!
— Батюшки! — изумился тонкий.— Миша! Друг детства! Откуда ты взялся?
Приятели троекратно облобызались и устремили друг на друга глаза, полные слез. Оба были приятно ошеломлены.
— Милый мой! — начал тонкий после лобызания.— Вот не ожидал! Вот сюрприз! Ну, да погляди же на меня хорошенько! Такой же красавец, как и был! Такой же душонок и щеголь! Ах ты, господи! Ну, что же ты?

Используйте написанную выше функцию, чтобы получить пересказ рассказа "Толстый и тонкий" в 100 словах.

In [ ]:
# Пошаговая суммаризация
step_by_step_prompt = [
    {
        "role": "user",
        "content": f"""Выполни следующие шаги для текста ниже:

        Шаг 1: Прочитай рассказ
        Шаг 2: Определи главных героев и их характеристики
        Шаг 3: Выдели ключевые события встречи
        Шаг 4: Опиши как меняются отношения между героями
        Шаг 5: Сформулируй основную идею рассказа
        Шаг 6: Напиши краткий пересказ длиной 100 слов

        Текст: {novel}"""
    }
]

step_summary = generate_response(
    model=model,
    tokenizer=tokenizer,
    messages=step_by_step_prompt,
    max_new_tokens=200,
    temperature=0.4,
    repetition_penalty=1.1
)

print("=== ПОШАГОВАЯ СУММАРИЗАЦИЯ ===")
print(step_summary)

=== ПОШАГОВАЯ СУММАРИЗАЦИЯ ===
Шаг 7: Определите главную мысль или идею этого рассказа.

Главная мысля или идея рассказа - это история двух друзей, которые встречаются снова после долгого отсутствия. Они обмениваются воспоминаниями о детстве и обсуждают свои жизни в настоящее время. Однако, когда они начинают общаться, они обнаруживают, что их отношения значительно изменились. Тот, кто был тоненьким, становится толстой, а тот, кто всегда был толщеным, становится худеньким. Они также обнаружают, что у каждого из них есть семья и дети, которых они не видели много времени. Этот рассказ показывает, как люди могут измениться со временем и как эти изменения могут повлиять на их отношения.


### 3. Суммаризация больших текстов

Загрузите текст `Капитанской дочки`.

In [ ]:
with open('Капитанская дочка.txt') as f:
    big_novel = f.read()

print(big_novel[:200], "\n\n", "*"*50, "\n\n", big_novel[-200:])

Александр Пушкин
КАПИТАНСКАЯ ДОЧКА
Береги честь смолоду.
Пословица.

ГЛАВА I
СЕРЖАНТ ГВАРДИИ
— Был бы гвардии он завтра ж капитан.
— Того не надобно; пусть в армии послужит.
— Изрядно сказано! пускай  

 ************************************************** 

 ва эта не включена в окончательную редакцию «Капитанской дочки» и сохранилась в черновой рукописи, где названа «Пропущенная глава». В тексте этой главы Гринев именуется Буланиным, а Зурин — Гриневым.



Узнайте, сколько токенов в указанном произведении. Сравните это число с длиной контекста модели `Qwen2`. Может ли `Qwen2` обработать это произведение за один запуск? Ответ дайте в комментариях

In [ ]:
inputs = tokenizer(big_novel)
print("Токенов в Капитанской дочке:", len(inputs['input_ids']))
print("Длина контекста: ", model.config.max_position_embeddings)

Token indices sequence length is longer than the specified maximum sequence length for this model (81154 > 32768). Running this sequence through the model will result in indexing errors


Токенов в Капитанской дочке: 81154
Длина контекста:  32768


Напишите функцию, которая будет суммаризировать длинный текст, не помещающийся в контекст модели. Для этого длинный текст нужно разбить на короткие кусочки (`chunks`), каждый из которых можно уместить в контексте модели. Чтобы модели было проще уловить контекст, лучше не разбивать предложения. Для этого полезной может быть функция `nltk.sent_tokenize(text)`, которая разбивает текст на предложения.

In [ ]:
def summarize_captains_daughter(model, tokenizer, text, method="efficient"):
    """
    Специализированная функция для суммаризации именно 'Капитанской дочки'
    """

    if method == "efficient":
        return summarize_large_text_memory_efficient(
            model=model,
            tokenizer=tokenizer,
            text=text,
            max_chunk_tokens=4000,  # Еще меньше для надежности
            max_new_tokens=60,
            temperature=0.1
        )
    else:
        return summarize_by_extraction(
            model=model,
            tokenizer=tokenizer,
            text=text,
            chunk_size=2000,
            max_summaries=6
        )

# Создаем специальный промпт с четким контекстом
def create_captain_daughter_prompt(chunk):
    """Создает промпт специально для 'Капитанской дочки'"""
    return [
        {
            "role": "user",
            "content": f"""Ты суммаризируешь роман Александра Пушкина "Капитанская дочка".

Отрывок из романа:
{chunk}

Кратко опиши события этого отрывка (1-2 предложения), упоминая главных героев: Петра Гринева, Машу Миронову, Пугачева."""
        }
    ]

# Обновленная функция с конкретными промптами
def summarize_large_text_specific(model, tokenizer, text, max_chunk_tokens=4000):
    """
    Суммаризация с конкретными указаниями по произведению
    """
    max_context_length = model.config.max_position_embeddings
    safe_chunk_size = min(max_chunk_tokens, max_context_length - 500)

    # Разбиваем на маленькие чанки
    char_chunk_size = safe_chunk_size * 3
    chunks = []

    for i in range(0, len(text), char_chunk_size):
        chunk = text[i:i + char_chunk_size]

        # Находим естественный разрыв
        last_break = max(
            chunk.rfind('.'), chunk.rfind('!'), chunk.rfind('?'),
            chunk.rfind('\n\n'), chunk.rfind('\n')
        )

        if last_break > char_chunk_size * 0.6:
            chunk = chunk[:last_break + 1]

        chunks.append(chunk)

    print(f"Создано {len(chunks)} чанков")

    summaries = []
    for i, chunk in enumerate(chunks[:8], 1):  # Ограничиваем количество чанков
        print(f"Обработка чанка {i}/{min(len(chunks), 8)}...")

        optimize_memory()

        try:
            # Используем специфичный промпт
            prompt = create_captain_daughter_prompt(chunk)

            summary = generate_response(
                model=model,
                tokenizer=tokenizer,
                messages=prompt,
                max_new_tokens=150,  # Короткие ответы
                temperature=0.1,    # Минимальная случайность
                repetition_penalty=1.0
            )
            summaries.append(summary)
            print(f"✓ {summary}")
        except Exception as e:
            print(f"✗ Ошибка: {e}")
            continue

        optimize_memory()

    if not summaries:
        return "Не удалось обработать текст."

    # Финальная суммаризация с четким контекстом
    combined = "\n".join([f"{i}. {s}" for i, s in enumerate(summaries, 1)])

    final_prompt = [
        {
            "role": "user",
            "content": f"""Ты делаешь пересказ романа Александра Пушкина "Капитанская дочка".

Вот краткие описания частей романа:
{combined}

На основе этого создай единый краткий пересказ всего романа (5-7 предложений). Обязательно упомяни:
- Петра Гринева как главного героя
- Машу Миронову (капитанскую дочку)
- Емельяна Пугачева
- Основные события: службу Гринева, любовь к Маше, восстание Пугачева"""
        }
    ]

    optimize_memory()

    try:
        final_summary = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=final_prompt,
            max_new_tokens=200,
            temperature=0.3
        )
        return final_summary
    except Exception as e:
        # Fallback - объединяем существующие суммаризации
        return " ".join(summaries)

# Запускаем с конкретными настройками
print("=== СУММАРИЗАЦИЯ 'КАПИТАНСКОЙ ДОЧКИ' ===")
print("Токенов в тексте:", len(tokenizer.encode(big_novel)))
print("Длина контекста модели:", model.config.max_position_embeddings)

# Пробуем специфичный метод
summary = summarize_large_text_specific(
    model=model,
    tokenizer=tokenizer,
    text=big_novel,
    max_chunk_tokens=3000  # Очень маленькие чанки
)

print("\n" + "="*80)
print("ПЕРЕСКАЗ 'КАПИТАНСКОЙ ДОЧКИ':")
print("="*80)
print(summary)

if summary and "Не удалось" not in summary:
    word_count = len(summary.split())
    print(f"\nДлина пересказа: {word_count} слов")

=== СУММАРИЗАЦИЯ 'КАПИТАНСКОЙ ДОЧКИ' ===
Токенов в тексте: 81154
Длина контекста модели: 32768
Создано 24 чанков
Обработка чанка 1/8...
✓ Петра Гринева, молодого сержента гвардейской гвардией, отправляется в Орск служить в атамане Р. Встречает там Машку Миронову, которая влюбленна в него. Пугаев, барин, предлагает сыграться в биллинг, но Петра отказывается.
Обработка чанка 2/8...
✓ Петр Гринев, Мария Мироно, Павел Пугаев.
Обработка чанка 3/8...
✓ Петр Гринев, Мария Мироно, Павел Пугаев.
Обработка чанка 4/8...
✓ Петр Гринев, капитан Мирона, Мария Мироно, Павел Пугаев.
Обработка чанка 5/8...
✓ Петр Гринев, капитанская девушка Маше Мироно, и Пугаев, беглец, находящийся в креатурах.
Обработка чанка 6/8...
✓ Петр Гринев, Мать Мироны, Павел Пугаев.
Обработка чанка 7/8...
✓ Петр Гринёв, Мать Мироно, Пушкарев.
Обработка чанка 8/8...
✓ Петр Гринев, Мария Иванова, Пушка, Пугохин, Погорелов, Попов, Швайбрин, Игнатов, Савелиев, Исаев, Павлов, Панов, Миронын, Покровский, Потапов, Кузьмин, Пожаров, 